<a href="https://colab.research.google.com/github/khaledn66/pyspark2/blob/main/29Aggregating_DataFrames_in_PySpark_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('agg').getOrCreate()
spark

In [11]:
!rm -rf pyspark2

In [12]:
!git clone https://github.com/khaledn66/pyspark2.git

Cloning into 'pyspark2'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 68 (delta 31), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 9.70 MiB | 6.31 MiB/s, done.
Resolving deltas: 100% (31/31), done.


## Read in the dataFrame for this Notebook

In [17]:
file_path = './pyspark2/nyc_air_bnb.csv'
bnb = spark.read.csv(file_path, inferSchema=True, header=True)
bnb.show(5)

+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|  host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|       John|           Brooklyn|   Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca...|   2845|   Jennifer|          Manhatt

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking.

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [19]:
df= bnb.limit(5).toPandas()
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Now print the schema so we can make sure all the variables have the correct types

In [20]:
print(bnb.printSchema())

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)

None


Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [25]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
df2 = bnb.withColumn("price",bnb["price"].cast(IntegerType())) \
.withColumn("minimum_nights",bnb["minimum_nights"].cast(IntegerType())) \
.withColumn("number_of_reviews", bnb["number_of_reviews"].cast(IntegerType())) \
.withColumn("reviews_per_month", bnb["reviews_per_month"].cast(IntegerType())) \
.withColumn("calculated_host_listings_count", bnb["calculated_host_listings_count"].cast(IntegerType()))
print(df2.printSchema())
df2.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [26]:
df2.count()

49079

### 2. How many total reviews does each host have?

In [27]:
df2.groupBy("host_id").agg(sum("number_of_reviews").alias("total_reviews")).show()

+-------+-------------+
|host_id|total_reviews|
+-------+-------------+
| 716306|          197|
|1203500|           35|
| 368528|            1|
|1577493|           16|
|1390555|           50|
|1317588|            3|
|2472680|          219|
|2155832|          266|
|2426404|            6|
|2740824|           22|
| 192750|            2|
|2682735|           50|
|4432173|           15|
|5959653|          240|
|4623093|           79|
|6390340|            0|
|4645357|          165|
|8422502|            2|
|9818634|            0|
|4906960|           65|
+-------+-------------+
only showing top 20 rows



### 3. Show the min and max of all the numeric variables in the dataset

In [28]:
df2.select("price","minimum_nights","number_of_reviews","reviews_per_month","calculated_host_listings_count").summary("min","max").show()

+-------+-----+--------------+-----------------+-----------------+------------------------------+
|summary|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|
+-------+-----+--------------+-----------------+-----------------+------------------------------+
|    min|  -74|             0|                0|                0|                             0|
|    max|10000|          1250|              629|               58|                           365|
+-------+-----+--------------+-----------------+-----------------+------------------------------+



### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [29]:
df2.groupBy("host_id").agg(sum("number_of_reviews").alias("total_reviews")).orderBy(desc("total_reviews")).show(1)

+--------+-------------+
| host_id|total_reviews|
+--------+-------------+
|37312959|         2273|
+--------+-------------+
only showing top 1 row



### 5. On average, how many nights did most hosts specify for a minimum?

In [30]:
df2.select(mean("minimum_nights")).show()

+-------------------+
|avg(minimum_nights)|
+-------------------+
| 7.1286126280910596|
+-------------------+



### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [31]:
df2.groupBy("neighbourhood").agg(avg("price").alias("avg_price")).orderBy(desc("avg_price")).show(1)

+--------------+---------+
| neighbourhood|avg_price|
+--------------+---------+
|Fort Wadsworth|    800.0|
+--------------+---------+
only showing top 1 row



### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [32]:
df2.filter("room_type IN ('Private room','Shared room') AND neighbourhood_group IN ('Manhattan','Brooklyn')").groupBy("room_type","neighbourhood_group").agg(avg("price").alias("avg_price")).show()

+------------+-------------------+------------------+
|   room_type|neighbourhood_group|         avg_price|
+------------+-------------------+------------------+
|Private room|          Manhattan|116.05400302114803|
| Shared room|          Manhattan| 89.06903765690376|
| Shared room|           Brooklyn| 50.52784503631961|
|Private room|           Brooklyn| 76.47234042553191|
+------------+-------------------+------------------+



### Alright that's all folks!

### Great job!